In [34]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import sys
sys.path.append('../creme/')
import custom_model
import shuffle
import creme
import utils
import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pyranges as pr
import shutil
import pickle
import kipoiseq
from itertools import combinations
import scipy
import os
import gene as bgene
import shuffle
from scipy.stats import pearsonr
import tensorflow_hub as hub


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
import tensorflow as tf
class Enformer:

  def __init__(self):
    tfhub_url = 'https://tfhub.dev/deepmind/enformer/1'
    os.environ['TFHUB_CACHE_DIR'] = '.'
    self._model = hub.load(tfhub_url).model

  def predict_on_batch(self, inputs):
    predictions = self._model.predict_on_batch(inputs)
    return {k: v.numpy() for k, v in predictions.items()}

  @tf.function
  def contribution_input_grad(self, input_sequence,
                              target_mask, output_head='human'):
    input_sequence = input_sequence[tf.newaxis]

    target_mask_mass = tf.reduce_sum(target_mask)
    with tf.GradientTape() as tape:
      tape.watch(input_sequence)
      prediction = tf.reduce_sum(
          target_mask[tf.newaxis] *
          self._model.predict_on_batch(input_sequence)[output_head]) / target_mask_mass

    input_grad = tape.gradient(prediction, input_sequence) * input_sequence
    input_grad = tf.squeeze(input_grad, axis=0)
    return tf.reduce_sum(input_grad, axis=-1)

In [71]:
model = custom_model.Enformer()
# model = Enformer()

In [76]:
seq_parser = utils.SequenceParser('../data/GRCh38.primary_assembly.genome.fa')
x = seq_parser.extract_seq_centered('chr1', 20000000, '+', 196608)

In [58]:
# predictions = model.predict_on_batch(x[np.newaxis])['human'][0]
predictions = model.predict(x[np.newaxis])[0]

predictions.shape

(896, 5313)

In [59]:
target_mask = np.zeros_like(predictions)
for idx in [447, 448]:
  # target_mask[idx, 4828] = 1
  target_mask[idx, 5111] = 1

In [78]:
x.shape

(196608, 4)

In [79]:
x = np.pad(x[np.newaxis], ((0, 0), (model.pseudo_pad // 2, model.pseudo_pad // 2), (0, 0)), 'constant')

contribution_scores = model.contribution_input_grad(x.astype(np.float32), target_mask).numpy()


In [75]:
contribution_scores

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)